## Connect to your workspace

In [23]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print("Ready to use Azure ML {} to work with {}".format(azureml.core.VERSION, ws.name))

Ready to use Azure ML 1.46.0 to work with ELP_test


## Train and register a model

Now let's train and register a model to deploy in a batch inferencing pipeline.

In [60]:
from azureml.core import Experiment, Model

# Create an Azure ML experiment in your workspace
experiment = Experiment(workspace=ws, name='mslearn-train-diabetes')
run = experiment.start_logging(snapshot_directory=None)
print("Starting experiment:", experiment.name)

KeyboardInterrupt: 

In [29]:
import os
# Create a folder for the pipeline step files
experiment_folder = 'pipeline'
os.makedirs(experiment_folder, exist_ok=True)

print(experiment_folder)

pipeline


In [44]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "your-compute-cluster"

try:
    # Check for existing compute target
    pipeline_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # If it doesn't already exist, create it
    try:
        compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2', max_nodes=2)
        pipeline_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
        pipeline_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)

Found existing cluster, use it.


In [67]:
from azureml.core import Environment
from azureml.core.runconfig import RunConfiguration
from azureml.core import Workspace, Dataset


# Create a Python environment for the experiment (from a .yml file)
experiment_env = Environment.from_conda_specification("experiment_env", experiment_folder + "/experiment_env.yml")

# Register the environment 
experiment_env.register(workspace=ws)
registered_env = Environment.get(ws, 'experiment_env')

# Create a new runconfig object for the pipeline
pipeline_run_config = RunConfiguration()

# Use the compute you created above. 
pipeline_run_config.target = pipeline_cluster

# Assign the environment to the run configuration
pipeline_run_config.environment = registered_env

print ("Run configuration created.")

Run configuration created.


In [68]:
from azureml.data import OutputFileDatasetConfig
from azureml.pipeline.steps import PythonScriptStep
from azureml.core import Workspace, Dataset


# Get the training dataset
dataset = Dataset.get_by_name(ws, name='batch-data') # how get data there remember
dataset.download(target_path='.', overwrite=True)

# Create an OutputFileDatasetConfig (temporary Data Reference) for data passed from step 1 to step 2
prepped_data = OutputFileDatasetConfig("prepped_data")

# Step 1, Run the data prep script
prep_step = PythonScriptStep(name = "Prepare Data",
                                source_directory = experiment_folder,
                                script_name = "prep.py",
                                arguments = ['--input-data', dataset.as_named_input('raw_data'),
                                             '--prepped-data', prepped_data],
                                compute_target = pipeline_cluster,
                                runconfig = pipeline_run_config,
                                allow_reuse = True)

# Step 2, run the training script
train_step = PythonScriptStep(name = "Train and Register Model",
                                source_directory = experiment_folder,
                                script_name = "train.py",
                                arguments = ['--training-data', prepped_data.as_input()],
                                compute_target = pipeline_cluster,
                                runconfig = pipeline_run_config,
                                allow_reuse = True)

print("Pipeline steps defined")

Pipeline steps defined


In [69]:
from azureml.core import Experiment
from azureml.pipeline.core import Pipeline
from azureml.widgets import RunDetails

# Construct the pipeline
pipeline_steps = [prep_step, train_step]
pipeline = Pipeline(workspace=ws, steps=pipeline_steps)
print("Pipeline is built.")

# Create an experiment and run the pipeline
experiment = Experiment(workspace=ws, name = 'mslearn-pipeline')
pipeline_run = experiment.submit(pipeline, regenerate_outputs=True)
print("Pipeline submitted for execution.")
RunDetails(pipeline_run).show()
pipeline_run.wait_for_completion(show_output=True)

Pipeline is built.
Created step Prepare Data [a3728aa1][39031545-aca3-4467-9fa0-9841471149f2], (This step will run and generate new outputs)
Created step Train and Register Model [257d50b4][a90f9c35-919b-4b94-af2c-8145b5503094], (This step will run and generate new outputs)
Submitted PipelineRun 2e4d6b75-10cc-47f8-8a20-bffe388663d3
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/2e4d6b75-10cc-47f8-8a20-bffe388663d3?wsid=/subscriptions/0aa54ba3-0284-4c54-8170-18cdcbefdf5f/resourcegroups/ELP/workspaces/ELP_test&tid=875c414e-5d00-4cdb-b77a-deae5d6ab201
Pipeline submitted for execution.


_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

PipelineRunId: 2e4d6b75-10cc-47f8-8a20-bffe388663d3
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/2e4d6b75-10cc-47f8-8a20-bffe388663d3?wsid=/subscriptions/0aa54ba3-0284-4c54-8170-18cdcbefdf5f/resourcegroups/ELP/workspaces/ELP_test&tid=875c414e-5d00-4cdb-b77a-deae5d6ab201
PipelineRun Status: Running


Expected a StepRun object but received <class 'azureml.core.run.Run'> instead.
This usually indicates a package conflict with one of the dependencies of azureml-core or azureml-pipeline-core.
Please check for package conflicts in your python environment


ActivityFailedException: ActivityFailedException:
	Message: Activity Failed:
{
    "error": {
        "code": "UserError",
        "message": "{\"NonCompliant\":\"Process '/azureml-envs/azureml_fdc05f88918fa818e3e93e934bd8223d/bin/python' exited with code 1 and error message 'Execution failed. Process exited with status code 1. Error: Traceback (most recent call last):\\n  File \\\"prep.py\\\", line 19, in <module>\\n    df = pd.read_csv(\\\"/home/saim/Desktop/abor\\\", sep='\\\\t')\\n  File \\\"/azureml-envs/azureml_fdc05f88918fa818e3e93e934bd8223d/lib/python3.8/site-packages/pandas/util/_decorators.py\\\", line 211, in wrapper\\n    return func(*args, **kwargs)\\n  File \\\"/azureml-envs/azureml_fdc05f88918fa818e3e93e934bd8223d/lib/python3.8/site-packages/pandas/util/_decorators.py\\\", line 317, in wrapper\\n    return func(*args, **kwargs)\\n  File \\\"/azureml-envs/azureml_fdc05f88918fa818e3e93e934bd8223d/lib/python3.8/site-packages/pandas/io/parsers/readers.py\\\", line 950, in r